# Purpose
This program generates a list of randomized perturbations of various parameters of a rocket's design, launch, and trajectory, then simulates a trajectory for each one and tabulates the perturbations and their final impact points. Impact points are in Cartesian coordinates, so columns are created for latitude, longitude, and height, and then the results are saved as a .csv file for later analysis by another program.

In order, the perturbations are as follows (in order):

| Column name |  Description | Units |
| ------- | --- | --- |
|'deg N' | Initial latitude offset | decimal degrees|
|'deg E' | Initial longitude offset| decimal degrees|
|'Launch Az'| Launcher azimuth| degrees |
|'Launch El'| Launcher elevation|  degrees|
|'Tip-Off'| Tip-off error is simulated | boolean|
|'Thrust Pitch'| Engine thrust vector misalignment, pitch | degrees|
|'Thrust Yaw'|Engine thrust vector misalignment, yaw | degrees|
|'mdot'| Mass flow rate| kg/s|
|'Ve'| Nozzle exit velocity | m/s |
|'mass'| Excess mass | positive number, kg |
|'drag'| Drag coefficient | dimensionless |
|'Wind'| Wind velocity is included| boolean|

I tried to pick reasonable estimates for standard deviations of these parameters. However, we still need engineers to select values reflective of what we can expect IRL.

In [1]:
%run Trajectory_Simulation.ipynb
import pandas as pd

/home/cory/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
amount = 100 # number of simulations per run
runs   = 5
# perturbations 'deg N', 'deg E', 'Launch Az', 'Launch El', 'Tip-Off',
            #   'Thrust Pitch', 'Thrust Yaw', 'mdot', 'Ve',
            #    'mass', 'drag', 'Wind'

def mc_init(num_sims):
    results = []
    perturbation_list = []

    for i in range(num_sims):
        perturbation = []
        perturbation.append(np.random.normal(0, 0.001)) # 111 m std dev 
        perturbation.append(np.random.normal(0, 0.001)) # 111 m std dev 
        perturbation.append(np.random.normal(AZ_PERTURB, 0.333333)) # degrees
        perturbation.append(np.random.normal(EL_PERTURB, 0.333333)) # degrees
        perturbation.append(np.random.rand() < 0.5)
        perturbation.append(np.random.normal(0, 0.0333333)) # degrees
        perturbation.append(np.random.normal(0, 0.0333333)) # degrees
        perturbation.append(np.random.normal(0, 0.02))
        perturbation.append(np.random.normal(0, 0.01))
        perturbation.append(np.random.exponential(2.5))
        perturbation.append(np.random.normal(0, 0.01))
        perturbation.append(np.random.rand() < 0.5)
        
        perturbation_list.append(perturbation)
        
    print('random perturbations obtained\n')
    
    for i, perturbation in enumerate(perturbation_list):
        if i % 10 == 0: print('iterations:', i)
        sim = trajectory(fin_staging, stage_root, stage_tip, stage_sweep, stage_span, stage_thickness, mass_red, M_PROP, MDOT, P_E,
               THROTTLE_WINDOW, MIN_THROTTLE,
               RCS_MDOT, RCS_P_E, RCS_P_CH,
               BALLAST, FIN_ROOT, FIN_TIP, FIN_SWEEP_ANGLE, FIN_SEMISPAN, FIN_THICKNESS, CON_NOSE_L,
                LOX_TANK_P, IPA_TANK_P, RIB_T, NUM_RADL_DVSNS,
               AIRFRM_IN_RAD, IPA_WT, OF, ENG_P_CH, ENG_T_CH, ENG_KE, ENG_MM,
                          perturbation, 
                          0.025, True, 0.005, True, True)
        x, y, z = sim.raw_states[-1][0][1]
        perturbation.append(x)
        perturbation.append(y)
        perturbation.append(z)
        
        results.append(perturbation)
    print('done simulations!\n')
    
    return results

In [3]:
def lat_long(df):
    array = df.loc[:, ['x', 'y', 'z']]
    X = [x for x in array['x']]
    Y = [y for y in array['y']]
    Z = [z for z in array['z']]
    coords = [Environment(None, 1).ECEF_to_geodetic([X[i], Y[i], Z[i]])
                                   for i in range(len(array))]
    df['lat'] = [coord[0] for coord in coords]
    df['long'] = [coord[1] for coord in coords]
    df['height'] = [coord[2] for coord in coords]
    
def handle_data(results):
    # landing coordinates
    results_df = pd.DataFrame.from_records(results,
                                           columns=['deg N', 'deg E', 'Launch Az', 'Launch El', 'Tip-Off',
                                                    'Thrust Pitch', 'Thrust Yaw', 'mdot', 'Ve',
                                                    'mass', 'drag', 'Wind',
                                                    'x', 'y', 'z'])
    lat_long(results_df)
    print(results_df.describe())
    return results_df

In [ ]:
for i in range(runs):
    results = mc_init(num_sims=amount)
    sim_data = handle_data(results)
    sim_data.to_csv(path_or_buf='./dispersion_sample_data/sim_data_' + str(i+4) + '.csv')

random perturbations obtained

iterations: 0
iterations: 10
iterations: 20
iterations: 30
iterations: 40
iterations: 50
iterations: 60
iterations: 70
